In [1]:
import pandas as pd
import os
import csv
import json

In [2]:
import random
import string

def generate_random_alphanumeric(length):
    """
    Generates a random alphanumeric string of a specified length.
    """
    # Define the characters to choose from (digits and uppercase letters)
    characters = string.ascii_uppercase + string.digits
    
    # Use random.choices to select characters and ''.join to create the string
    alphanumeric_string = ''.join(random.choices(characters, k=length))
    
    return alphanumeric_string

In [3]:
output_columns = ['*Sale Order Number', '*Pickup Location Name', '*Transport Mode',
       '*Payment Mode', 'COD Amount', '*Customer Name', '*Customer Phone',
       '*Shipping Address Line1', 'Shipping Address Line2', '*Shipping City',
       '*Shipping State', '*Shipping Pincode', '*Item Sku Code',
       '*Item Sku Name', '*Quantity Ordered', 'Packaging Type',
       '*Unit Item Price', 'Length (cm)', 'Breadth (cm)', 'Height (cm)',
       'Weight (gm)', 'Fragile Shipment', 'Discount Type', 'Discount Value',
       'Tax Class Code', 'Customer Email',
       'Billing Address same as Shipping Address', 'Billing Address Line1',
       'Billing Address Line2', 'Billing City', 'Billing State',
       'Billing Pincode', 'e-Way Bill Number', 'Seller Name',
       'Seller GST Number', 'Seller Address Line1', 'Seller Address Line2',
       'Seller City', 'Seller State', 'Seller Pincode']

In [4]:
input_path = r"C:\Users\silla\Downloads\orders_export_28_10_v2.csv"
output_path = r"C:\Users\silla\Downloads\Bulk_Order_Upload_28_10_v2.csv"


In [5]:
input_df = pd.read_csv(input_path)
grouped_orders = input_df.groupby('Name')

In [12]:
group_order_dict = {}
for order_number, order_data in grouped_orders:
    group_order_dict[order_number] = {
        "Financial Status": order_data['Financial Status'].iloc[0] if not pd.isna(order_data['Financial Status'].iloc[0]) else "N/A",
        "Subtotal": order_data['Subtotal'].iloc[0] if not pd.isna(order_data['Subtotal'].iloc[0]) else "N/A",
        "Total": order_data['Total'].iloc[0] if not pd.isna(order_data['Total'].iloc[0]) else "N/A",
        "Shipping": order_data['Shipping'].iloc[0] if not pd.isna(order_data['Shipping'].iloc[0]) else "N/A",
        "Discount Amount": order_data['Discount Amount'].iloc[0] if not pd.isna(order_data['Discount Amount'].iloc[0]) else "N/A",
        "Shipping Name": order_data['Shipping Name'].iloc[0] if not pd.isna(order_data['Shipping Name'].iloc[0]) else "N/A",
        "Shipping Phone": str(order_data['Shipping Phone'].iloc[0] if not pd.isna(order_data['Shipping Phone'].iloc[0]) else "N/A").replace(".0","")[-10:],
        "Shipping Street": order_data['Shipping Street'].iloc[0] if not pd.isna(order_data['Shipping Street'].iloc[0]) else "N/A",
        "Shipping City": order_data['Shipping City'].iloc[0] if not pd.isna(order_data['Shipping City'].iloc[0]) else "N/A",
        "Shipping Province": order_data['Shipping Province'].iloc[0] if not pd.isna(order_data['Shipping Province'].iloc[0]) else "N/A",
        "Shipping Zip": order_data['Shipping Zip'].iloc[0] if not pd.isna(order_data['Shipping Zip'].iloc[0]) else "N/A",
    }
    partial_payment = 0
    for idx, row in order_data.iterrows():
        if "Partial Payment" in row['Lineitem name']:
            partial_payment += row['Lineitem price']
    group_order_dict[order_number]['Partial Payment'] = partial_payment

In [13]:
group_order_dict

{'#3333': {'Financial Status': 'paid',
  'Subtotal': np.float64(1014.6),
  'Total': np.float64(1014.6),
  'Shipping': np.float64(0.0),
  'Discount Amount': np.float64(53.4),
  'Shipping Name': 'Faisal Ahamed',
  'Shipping Phone': '9788165082',
  'Shipping Street': 'New Street, New street',
  'Shipping City': 'Muthupet',
  'Shipping Province': 'TN',
  'Shipping Zip': "'614704",
  'Partial Payment': 0},
 '#3340': {'Financial Status': 'paid',
  'Subtotal': np.float64(367.65),
  'Total': np.float64(466.65),
  'Shipping': np.float64(99.0),
  'Discount Amount': np.float64(19.35),
  'Shipping Name': 'TUSHAR TIBILE',
  'Shipping Phone': '7709428621',
  'Shipping Street': 'Shivshankar vasahat, Shree hari krupa',
  'Shipping City': 'Kurundwad',
  'Shipping Province': 'MH',
  'Shipping Zip': "'416106",
  'Partial Payment': 0},
 '#3342': {'Financial Status': 'paid',
  'Subtotal': np.float64(669.75),
  'Total': np.float64(768.75),
  'Shipping': np.float64(99.0),
  'Discount Amount': np.float64(35.2

In [16]:
with open(input_path, 'r', encoding='utf-8', errors='ignore', newline='') as infile, open(output_path, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=output_columns)
    writer.writeheader()

    for row in reader:
        transformed_dict = {}
        order_number = row["Name"]
        for column in output_columns:
            if column == "*Sale Order Number":
                transformed_dict[column] = row["Name"].replace("#","")
            elif column == "*Pickup Location Name":
                transformed_dict[column] = "Sillage"
            elif column == "*Transport Mode":
                transformed_dict[column] = "Express"
            elif column == "*Payment Mode":
                if group_order_dict[order_number].get("Financial Status", "") == "paid":
                    transformed_dict[column] = "Prepaid"
                else:
                    transformed_dict[column] = "COD"
            elif column == "COD Amount":
                if group_order_dict[order_number].get("Financial Status", "") != "paid":
                    transformed_dict[column] = group_order_dict[order_number]['Total'] - group_order_dict[order_number]['Shipping'] - (2*group_order_dict[order_number]['Partial Payment'])
                else:
                    transformed_dict[column] = ""
            elif column == "*Customer Name":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Name"]
            elif column == "*Customer Phone":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Phone"]
            elif column == "*Shipping Address Line1":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Street"]
            elif column == "Shipping Address Line2":
                transformed_dict[column] = ""
            elif column == "*Shipping City":
                transformed_dict[column] = group_order_dict[order_number]["Shipping City"]
            elif column == "*Shipping State":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Province"]
            elif column == "*Shipping Pincode":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Zip"].replace("'","")
            elif column == "*Item Sku Code":
                transformed_dict[column] = generate_random_alphanumeric(7)
            elif column == "*Item Sku Name":
                transformed_dict[column] = " ".join(row["Lineitem name"].split()[:2])
            elif column == "*Quantity Ordered":
                transformed_dict[column] = row["Lineitem quantity"]
            elif column == "Packaging Type":
                transformed_dict[column] = "Plastic cover/Flyer"
            elif column == "*Unit Item Price":
                transformed_dict[column] = row["Lineitem price"]
            elif column == "Length (cm)":
                transformed_dict[column] = "18"
            elif column == "Breadth (cm)":
                transformed_dict[column] = "8"
            elif column == "Height (cm)":
                transformed_dict[column] = "25"
            elif column == "Weight (gm)":
                transformed_dict[column] = "450"
            elif column == "Fragile Shipment":
                transformed_dict[column] = "Yes"
            elif column == "Customer Email":
                transformed_dict[column] = row['Email']
            elif column == "Billing Address same as Shipping Address":
                transformed_dict[column] = "Yes"
            else:
                transformed_dict[column] = ""
        
        writer.writerow(transformed_dict)
            
            
            
            
                
            
            
            
                 

In [8]:
with open(input_path, 'r', encoding='utf-8', errors='ignore', newline='') as infile, open(output_path, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=output_columns)
    writer.writeheader()

    for row in reader:
        transformed_dict = {}
        order_number = row["Name"]
        for column in output_columns:
            if column == "*Sale Order Number":
                transformed_dict[column] = row["Name"].replace("#","")
            elif column == "*Pickup Location Name":
                transformed_dict[column] = "Sillage"
            elif column == "*Transport Mode":
                transformed_dict[column] = "Express"
            elif column == "*Payment Mode":
                if group_order_dict[order_number].get("Financial Status", "") == "paid":
                    transformed_dict[column] = "Prepaid"
                else:
                    transformed_dict[column] = "COD"
            elif column == "COD Amount":
                if group_order_dict[order_number].get("Financial Status", "") != "paid":
                    transformed_dict[column] = group_order_dict[order_number]['Total']
                else:
                    transformed_dict[column] = ""
            elif column == "*Customer Name":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Name"]
            elif column == "*Customer Phone":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Phone"]
            elif column == "*Shipping Address Line1":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Street"]
            elif column == "Shipping Address Line2":
                transformed_dict[column] = ""
            elif column == "*Shipping City":
                transformed_dict[column] = group_order_dict[order_number]["Shipping City"]
            elif column == "*Shipping State":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Province"]
            elif column == "*Shipping Pincode":
                transformed_dict[column] = group_order_dict[order_number]["Shipping Zip"].replace("'","")
            elif column == "*Item Sku Code":
                transformed_dict[column] = generate_random_alphanumeric(7)
            elif column == "*Item Sku Name":
                transformed_dict[column] = " ".join(row["Lineitem name"].split()[:2])
            elif column == "*Quantity Ordered":
                transformed_dict[column] = row["Lineitem quantity"]
            elif column == "Packaging Type":
                transformed_dict[column] = "Plastic cover/Flyer"
            elif column == "*Unit Item Price":
                transformed_dict[column] = row["Lineitem price"]
            elif column == "Length (cm)":
                transformed_dict[column] = "18"
            elif column == "Breadth (cm)":
                transformed_dict[column] = "8"
            elif column == "Height (cm)":
                transformed_dict[column] = "25"
            elif column == "Weight (gm)":
                transformed_dict[column] = "450"
            elif column == "Fragile Shipment":
                transformed_dict[column] = "Yes"
            elif column == "Customer Email":
                transformed_dict[column] = row['Email']
            elif column == "Billing Address same as Shipping Address":
                transformed_dict[column] = "Yes"
            else:
                transformed_dict[column] = ""
        
        writer.writerow(transformed_dict)
            
            
            
            
                
            
            
            
                 